In [6]:
# Import libraries
import numpy as np
import pandas as pd
import scipy as sp
import argparse
import shap
import torch
from torch.serialization import save
from transformers_interpret import SequenceClassificationExplainer
import matplotlib.pyplot as plt

from kmembert.models import HealthBERT
from kmembert.utils import create_session

In [2]:
# Import argparse
import argparse
parser = argparse.ArgumentParser()
parser.add_argument("-d", "--data_folder", type=str, default="data/ehr/test.csv", 
    help="data path to access to the testing file")
parser.add_argument("-p", "--path_dataset", type=str, default="data/ehr/test.csv", 
    help="data path to access to the testing file")
parser.add_argument("-r", "--resume", type=str, default="kmembert-base", 
    help="result folder in with the saved checkpoint will be reused")
parser.add_argument("-nr", "--nrows", type=int, default=10, 
    help="maximum number of samples for testing")
parser.add_argument("-f", "--folder_to_save", type=str, default="graphs", 
    help="folder to save the figures")
parser.add_argument("-ng", "--noigr", type=int, default=2, 
    help="The Noigr of a patient")
args = parser.parse_args("")

In [3]:
# Load Model
_, _, device, config = create_session(args)
model = HealthBERT(device, config)

> DEVICE:  cpu
> ROOT:    c:\Users\DIPIAZZA\Documents\CLBProjet\VirtualMachine_T2\KmemBERT
> SESSION: c:\Users\DIPIAZZA\Documents\CLBProjet\VirtualMachine_T2\KmemBERT\results\ipykernel_launcher_22-06-02_14h14m07s

Using mode density (Health BERT checkpoint kmembert-base)

Loading camembert and its tokenizer...
if config.resume from health_bert.py
Resuming with model at kmembert-base...
Successfully loaded



In [5]:
tokenizer = model.tokenizer
model = model.camembert

In [59]:
# Use cls_explainer to get Word importance
cls_explainer = SequenceClassificationExplainer(
    model,
    tokenizer)
txt_to_explain = "Elle est vue avec un scanner cervico-thoraco-abdomino-pelvien et une échographie cardiaque.  La tolérance du traitement est marquée par une asthénie qui a tendance à s'aggraver depuis le début du traitement et ce malgré la baisse de dose à 600 mg/j (pour un début à 800 mg/j). La patiente est obligée de se reposer environ 1 heure après le repas de midi. Par ailleurs, elle présente des nausées de grade 1 et des diarrhées de grade 1 avec un maximum de 3 à 4 selles liquides par jour. La patiente est par ailleurs peu symptomatique.  L'échographie cardiaque réalisée ce jour ne montre pas d'anomalie. La fraction d'éjection ventriculaire gauche est conservée à 58 %."
word_attributions = cls_explainer(txt_to_explain)

In [35]:
# Read medical vocabulary
import json
f = open("medical_voc/large.json", encoding='utf-8')
dictio = json.load(f)
med_voc = []
for i in range(len(dictio)):
    med_voc.append(dictio[i][0])

In [62]:
# Get a dict of words with attributions
new_word = {}
word = dict(word_attributions)

for k, v in word.items():
    # Check non alpha numeric character
    new_k = k
    if ("▁" in k) | ("_" in k):
        new_k = new_k.replace("▁", "")
        new_k = new_k.replace("_", "")
    new_word[new_k] = -v
new_word

{'<s>': -0.0,
 'Elle': -0.006227284458226978,
 'est': -0.004059863772618026,
 'vue': 0.01702365803697663,
 'avec': 0.022076172303687913,
 'un': 0.031558633712696814,
 'scanner': 0.008514533935637716,
 'ce': -0.00797642818053381,
 'rv': 0.011642149884808923,
 'ico': 0.00930713404883973,
 '-': -0.031647800545111404,
 'thor': -0.007585478514049288,
 'aco': -0.010471905736378993,
 'ab': -0.004042881982295822,
 'dom': -0.004532522835211056,
 'ino': -0.010236792605350811,
 'pelvien': -0.018197488576654215,
 'et': -0.027472333484562784,
 'une': 0.02300009717764583,
 'échographie': -0.005785011318624106,
 'cardiaque': -0.0029311966589572365,
 '': 0.008052187868537565,
 '.': 0.006136843948483574,
 'La': 0.02152029754119849,
 'tolérance': -0.49175216039545516,
 'du': -0.11147701556244888,
 'traitement': -0.13699200707042702,
 'marquée': -0.021455357481434977,
 'par': -0.06261560807905522,
 'asthénie': -0.00453361399790162,
 'qui': 0.012921057760306354,
 'a': 0.028774369803148674,
 'tendance': 0.

In [64]:
# Mise en forme pour affichage
txt = list(new_word.keys()) # Le texte
txt_data = (list(map(( lambda x: x+' '), txt)),)
val = list(new_word.values()) # Les valeurs
txt_values = np.array([val])

# Création de l'objet shap et affichage
test = shap._explanation.Explanation(values=txt_values)
test.data = txt_data
test.base_values = np.array([0.])
shap.plots.text(test[0])